In [1]:
%%bash
git clone https://github.com/pytorch/fairseq
cd fairseq
pip install --editable .

Obtaining file:///content/fairseq
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Running setup.py develop for fairseq


Cloning into 'fairseq'...


Кладем скрипты предпобработки и вашу модель по нужным папкам.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp -r '/content/gdrive/My Drive/Bidir translation/data-bin'\
    '/content/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mkdir /content/gdrive/My\ Drive/Bidir\ translation/checkpoints/transformer_en_de

In [0]:
%%bash
echo 'Cloning Moses github repository (for tokenization scripts)...'
git clone https://github.com/moses-smt/mosesdecoder.git

echo 'Cloning Subword NMT repository (for BPE pre-processing)...'
git clone https://github.com/rsennrich/subword-nmt.git

Cloning Moses github repository (for tokenization scripts)...
Cloning Subword NMT repository (for BPE pre-processing)...


Cloning into 'mosesdecoder'...
Cloning into 'subword-nmt'...


In [3]:
!pip install sacremoses

     |████████████████████████████████| 870kB 16.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=0ea92c9a23d36222cf2c41005bfcf70ca9b038335eedad012505074fd1aad5bc
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    data-bin/iwslt17.tokenized.en-de \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir /content/gdrive/My\ Drive/Bidir\ translation/checkpoints/transformer_en_de \
    --save-interval	5


2020-03-06 08:29:59 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer_iwslt_de_en', attention_dropout=0.0, best_checkpoint_metric='bleu', bpe=None, broadcast_buffers=False, bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin/iwslt17.tokenized.en-de', dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=4, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_po

In [0]:
fairseq-generate data-bin/iwslt17.tokenized.en-de \
    --path /content/gdrive/My\ Drive/Bidir\ translation/checkpoints/transformer_en_de/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe

In [0]:
idx = 0
while True:
  idx += 1
  if idx > 100000000:
    idx = 0